# Computations with Binomial Ideals

* Author: Wolfram Decker, Clara Petroll
* Version: Oscar version 1.0.0
* Last modified: May 15, 2024

This tutorial demonstrates how to compute a binomial primary decomposition of unital binomial ideals over $Q$.

A *binomial* is a polynomial consisting of at most two terms. A *binomial ideal*
is an ideal which can be generated by binomials. A *binomial primary decomposition*
is a primary decomposition of a binomial ideal into primary ideals which are binomial
as well. In this section, focusing on polynomial rings over fields, we discuss functionality
for computing such decompositions.

We begin by recalling that a proper ideal $I$ of a polynomial ring $K[x_1, \dots, x_n]$ over a
field $K$ is called *cellular* if each variable $x_i$ is either a nonzerodivisor or nilpotent modulo $I$.
In this case, we refer to the nonzerodivisors among the variables as the *cell variables* with respect to $I$.
A *cellular decomposition* of a proper binomial ideal $I$ of $K[x_1, \dots, x_n]$ is a decomposition of $I$ into cellular binomial ideals.
Using Noetherian induction, it is not too difficult to show that such decompositions exist. 

With this notation, the algorithms for computing binomial primary decompositions proceed in two main steps:

- First, compute a cellular decomposition of the given binomial ideal.
- Then, decompose each cellular component into primary binomial ideals.

While the first step can be performed over any ground field for which Gröbner bases are implemented, the second step may require a field extension: From a theoretical point of view, the existence of binomial primary decompositions is only guaranteed if the ground field is algebraically closed.

## Terminology

A *pure difference binomial* is a binomial which is the difference of two monomials. A *unital binomial ideal* is an ideal which can be generated by pure difference binomials and monomials. Note that cellular components of unital binomial ideals are unital as well. For unital binomial ideals in $\mathbb Q[x_1, \dots, x_n]$, binomial primary decompositions exist already over cyclotomic extensions of $\mathbb Q$. In particular, any such ideal can be decomposed over the abelian closure $\mathbb Q^{\text{ab}}$ of $\mathbb Q$. While OSCAR offers functionality for doing this, computing  binomial primary decompositions  in other cases is not yet supported. 

See the [number theory](https://docs.oscar-system.org/stable/NumberTheory/abelian_closure/) chapter for how to deal with $\mathbb Q^{\text{ab}}$.




For further imformation see
* David Eisenbud and Bernd Sturmfels, Binomial ideals, Duke Math. J., 84 (1996), No. 1, 1-45.
* Thomas Kahle, Decompositions of binomial ideals, Ann. Inst. Statist. Math. 62 (2010), No. 4, 727-745.
* Clara Petroll, Primary decompositions of binomial ideals, Bachelor Thesis, TU Kaiserslautern (2017).

Content:
1. [Associated Primes](#1.-Associated-Primes)
2. [Cellular Decomposition](#2.-Cellular-Decomposition)
3. [Binomial Primary Decomposition](#3.-Binomial-Primary-Decomposition)

In [1]:
using Oscar

  ___   ____   ____    _    ____
 / _ \ / ___| / ___|  / \  |  _ \   |  Combining ANTIC, GAP, Polymake, Singular
| | | |\___ \| |     / _ \ | |_) |  |  Type "?Oscar" for more information
| |_| | ___) | |___ / ___ \|  _ <   |  Manual: https://docs.oscar-system.org
 \___/ |____/ \____/_/   \_\_| \_\  |  1.1.0-DEV #StuffForWolframTutorial 5aa97ef 2024-05-15


## 1. Associated Primes
For cellular ideals we can compute the associated primes and the minimal associated primes.
 

In [2]:
R, x = polynomial_ring(QQ, :x => 1:6)

(Multivariate polynomial ring in 6 variables over QQ, QQMPolyRingElem[x[1], x[2], x[3], x[4], x[5], x[6]])

In [3]:
I = ideal([x[5]*(x[1]-x[2]),x[6]*(x[3]-x[4]),x[5]^2,x[6]^2,x[5]*x[6]])

Ideal generated by
  x[1]*x[5] - x[2]*x[5]
  x[3]*x[6] - x[4]*x[6]
  x[5]^2
  x[6]^2
  x[5]*x[6]

The ideal is {$1,2,3,4$}-cellular.


In [4]:
is_cellular(I)

(true, [1, 2, 3, 4])

Now we compute the minimal associated primes, as well as the associated primes.


In [5]:
cellular_minimal_associated_primes(I)

1-element Vector{MPolyIdeal{AbstractAlgebra.Generic.MPoly{QQAbElem{AbsSimpleNumFieldElem}}}}:
 Ideal (x[5], x[6])

In [6]:
cellular_associated_primes(I)

3-element Vector{MPolyIdeal{AbstractAlgebra.Generic.MPoly{QQAbElem{AbsSimpleNumFieldElem}}}}:
 Ideal (x[5], x[6])
 Ideal (x[1] - x[2], x[5], x[6])
 Ideal (x[3] - x[4], x[5], x[6])

## 2. Cellular Decomposition
We consider a noncellular ideal in a polynomial ring with three variables.

In [7]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])

(Multivariate polynomial ring in 3 variables over QQ, QQMPolyRingElem[x, y, z])

In [8]:
I = ideal([x-y,x^3-1,z*y^2-z])

Ideal generated by
  x - y
  x^3 - 1
  y^2*z - z

In fact, the ideal is noncellular, which can be verified as follows:


In [9]:
is_cellular(I)

(false, [3])

The output means that the variable $z$ is a zerodivisor modulo $I$ but not nilpotent modulo $I$. Hence the ideal cannot be cellular. 

We now compute the decomposition of $I$ into cellular binomial ideals.

In [10]:
bcd = cellular_decomposition(I)

2-element Vector{MPolyIdeal{QQMPolyRingElem}}:
 Ideal (y - 1, x - 1)
 Ideal (x - y, x^3 - 1, y^2*z - z, z)

The intersection is equal to the original ideal.


In [11]:
bcd[1] ∩ bcd[2] == I

true

## 3. Binomial Primary Decomposition

To compute the primary decomposition of a binomial ideal $I$, the algorithm first decomposes $I$
into cellular ideals as above and then computes for each of the cellular binomial ideals a binomial primary decomposition.

In [12]:
binomial_primary_decomposition(I)

3-element Vector{Tuple{MPolyIdeal{AbstractAlgebra.Generic.MPoly{QQAbElem{AbsSimpleNumFieldElem}}}, MPolyIdeal{AbstractAlgebra.Generic.MPoly{QQAbElem{AbsSimpleNumFieldElem}}}}}:
 (Ideal (z, y - zeta(3), x - zeta(3)), Ideal (y - zeta(3), x - zeta(3), z))
 (Ideal (z, y + zeta(3) + 1, x + zeta(3) + 1), Ideal (y + zeta(3) + 1, x + zeta(3) + 1, z))
 (Ideal (y - 1, x - 1), Ideal (y - 1, x - 1, x*y - 1))